In [1]:
# Pandas dibutuhkan untuk manipulasi dan transformasi data
import pandas as pd

# Sklearn digunakan untuk segala hal yang berhubungan dengan machine learning modeling
# train_test_split : untuk membagi data train menjadi data untuk training dan validasi
from sklearn.model_selection import train_test_split

# DecisionTreeClassifier : contoh model yang digunakan disini yaitu Decision Tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import OneHotEncoder

In [2]:
# Data Preparation
df = pd.read_csv('sampling_healtkathon2022/sampling_healtkathon2022.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sampling_healtkathon2022/sampling_healtkathon2022.csv'

In [ ]:
# Data Cleaning

# Penghapusan Row yang ada NULL VALUE
TrueClean= df.dropna(how='any',axis=0) 
print(len(df))
print(len(TrueClean))


11401882
7183932


In [ ]:
# Duplikasi dataframe
Pred = TrueClean.drop(columns=['label'])

In [ ]:
# Pembuatan data training

satu = TrueClean.loc[TrueClean['label'] == 1] # Pengambilan yang labelnya 1
nolfull = TrueClean.loc[TrueClean['label'] == 0] # Pengambilan yang label 0



nol = nolfull.sample(n=len(satu)) # Pengambilan label 0 sejumlah banyaknya label 1



In [ ]:
rand_processed = satu.append(nol)
rand_processed = rand_processed.drop('id', axis=1)
gender_map = {"L":int(0), "P":int(1)}
rand_processed.replace({"jenkel": gender_map}, inplace=True)
rand_processed.replace({"jenkel": gender_map}, inplace=True)
rand_processed = rand_processed.drop('tgldatang', axis=1)
rand_processed = rand_processed.drop('tglpulang', axis=1)

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
rand_fixbet = ohe.fit_transform(rand_processed[['typefaskes']])
rand_processed[ohe.categories_[0]] = rand_fixbet
rand_processed = rand_processed.drop('typefaskes', axis=1)
rand_processed

In [ ]:
Train = rand_processed
Train = Train.sample(frac = 1)
Train.head()

,id,id_peserta,dati2,typefaskes,usia,jenkel,pisat,tgldatang,tglpulang,jenispel,...,biaya,jenispulang,cbg,kelasrawat,kdsa,kdsp,kdsr,kdsi,kdsd,label
3161009,3316213,6809171,178,SD,51,L,5.0,2018-04-20T17:00:00.000Z,2018-04-20T17:00:00.000Z,2,...,737700.0,1.0,N-3-15-0,3,None,None,None,None,None,0
8749112,4962036,4729899,301,C,31,P,1.0,2018-01-14T17:00:00.000Z,2018-01-14T17:00:00.000Z,2,...,196900.0,1.0,Q-5-42-0,3,None,None,None,None,None,1
8300914,6204949,5135336,90,I3,7,L,4.0,2018-01-01T17:00:00.000Z,2018-01-01T17:00:00.000Z,2,...,184300.0,1.0,Q-5-42-0,3,None,None,None,None,None,1
11348639,2441843,5437637,134,SC,26,L,4.0,2020-08-16T17:00:00.000Z,2020-08-16T17:00:00.000Z,2,...,132900.0,1.0,Q-5-40-0,3,None,None,None,None,None,0
10163639,4063732,7229133,27,B,39,P,1.0,2019-03-31T17:00:00.000Z,2019-03-31T17:00:00.000Z,2,...,198900.0,1.0,F-5-14-0,3,None,None,None,None,None,1


In [ ]:
print(type(nol))
print(len(satu))
print(len(Train))

<class 'pandas.core.frame.DataFrame'>
61846
123692


In [ ]:
X = Train.drop(columns=['label'])
y = Train.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Evaluation

y_true = [1,1,1,1,1,1,1,1,1,1]
y_pred = [0,1,1,1,1,1,1,1,1,1]


def check(true,pred)  :
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    Accuracy = (tn+tp) / (tn+fp+tp+fn)
    Precision = tp/(tp+fp)
    Recall = tp/(tp+fn)
    Specifity = tn/(tn+fp)  

    print("Accuracy    :", Accuracy, "\nPrecision   :", Precision, "\nRecall      :", Recall, "\nSpecifity   :", Specifity)


check(y_true,y_pred)

